In [33]:

#import numpy as np
import splat.simulate as spsim
import splat.photometry as sphot
import splat.core as spl
import splat.empirical as spem
import splat.simulate as spsim
import splat.evolve as spev
import bisect
import astropy.units as u
import scipy
import numpy as np

import emcee
#import warnings; warnings.simplefilter('ignore')
from astropy.coordinates import SkyCoord


def lnprob(d, dmax, dmin, coord):
    """
    probability density in all directions
    """
    if (d>dmax) or (d<dmin) :
        return -np.inf
    else:
        new_coord=SkyCoord(ra=coord.ra, dec=coord.dec, distance=d*u.pc)
        r=(new_coord.cartesian.x**2+new_coord.cartesian.y**2)**0.5
        return np.log((d**3)*spsim.galactic_density_juric(r, new_coord.cartesian.z, unit=u.pc, center='sun').value)

In [26]:
import wisps.simulations as wispsim

In [11]:
coords=wispsim.OBSERVED_POINTINGS

In [50]:
%%capture
ndim=1
nwalkers = 250
p0 = np.random.rand(ndim * nwalkers).reshape((nwalkers, ndim))
sampler = emcee.EnsembleSampler(nwalkers, ndim, lnprob, args=(300.0, 20.0, coords[0]))
sampler.run_mcmc(p0, 1000)

In [51]:
import matplotlib.pyplot as plt
%matplotlib inline

In [52]:
ds=sampler.flatchain[:, 0]

In [53]:
np.concatenate(sampler.lnprobability)

array([-inf, -inf, -inf, ..., -inf, -inf, -inf])

In [54]:
ds.shape

(250000,)

In [55]:
ds[~np.isinf(np.concatenate(sampler.lnprobability))]

array([], dtype=float64)